In [ ]:
%pip install llama-index llama-index-llms-mistralai llama-index-embeddings-fastembed -q

In [ ]:
import os
import getpass

os.environ["MISTRAL_API_KEY"] = getpass.getpass('Enter Mistral API Key')

In [ ]:
from llama_index.core import Settings
from llama_index.llms.mistralai import MistralAI 
from llama_index.embeddings.fastembed import FastEmbedEmbedding

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    download_loader,
    RAKEKeywordTableIndex,
)

In [ ]:
Settings.llm = MistralAI(model="mistral-small-latest")
Settings.embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en")

In [ ]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

In [ ]:
reader = SimpleDirectoryReader(input_files=["./data/10k/lyft_2021.pdf"])
data = reader.load_data()

In [ ]:
index = VectorStoreIndex.from_documents(data)

In [ ]:
query_engine = index.as_query_engine(streaming=True, similarity_top_k=3)

In [ ]:
response = query_engine.query(
    "What was the impact of COVID? Show statements in bullet form and show"
    " page reference after each statement."
)
#response.print_response_stream()

In [ ]:
for node in response.source_nodes:
    print("-----")
    text_fmt = node.node.get_content().strip().replace("\n", " ")[:1000]
    print(f"Text:\t {text_fmt} ...")
    print(f"Metadata:\t {node.node.metadata}")
    print(f"Score:\t {node.score:.3f}")